有时损失函数非常的复杂，我们不能保证算出的梯度表达式是正确的,
因此，我们有必要先根据梯度的定义来求出梯度,进而来验证我们使用的梯度表达式是否正确

那为什么不直接用梯度的定义来求梯度呢？
因为用最原始的梯度的定义来求解梯度，运算量比较大

In [1]:
import numpy as np
np.random.seed(666)
X = np.random.random(size=(1000,10))
true_theta = np.arange(1,12,dtype = float)
Xb = np.hstack([np.ones(shape = (X.shape[0],1)),X])
y = Xb.dot(true_theta) + np.random.normal(size = 1000)

In [2]:
# 用之前推导的批量梯度下降法
def J(y,Xb,theta):
    try:
        return (y - Xb.dot(theta)).T.dot(y - Xb.dot(theta))/len(y)
    except:
        return float('inf')

def dJ_math(y,Xb,theta):
    return Xb.T.dot(Xb.dot(theta)-y)*2/len(y)


In [3]:
# 利用定义编写 梯度函数
def dJ_debug(y,Xb,theta,interval = 0.01):
    theta_len = len(theta)
    dJ_theta = np.empty(theta_len)
    for i in range(theta_len):
        theta1 = theta.copy()
        theta1[i] += interval
        theta2 = theta.copy()
        theta2[i] -= interval
        dJ_theta[i] = (J(y,Xb,theta1) - J(y,Xb,theta2))/(2*interval)
    return dJ_theta        

In [4]:
# 由于有两个梯度函数，故，我们将梯度函数作为参数，以便后续选取
def gradient_descent(dJ,y,Xb,theta,eta,epsilon = 1e-8,iters = 1e3):
    i_iters = 0
    while i_iters < iters:
        i_iters +=1
        last_theta = theta
        theta = theta - eta*dJ(y,Xb,theta)
        if np.abs(J(y,Xb,theta) - J(y,Xb,last_theta))<epsilon:
            break
    return theta

In [5]:
theta = np.zeros(Xb.shape[1])
eta = 0.1
%time theta_hat = gradient_descent(dJ_debug,y,Xb,theta,eta)
theta_hat

CPU times: user 952 ms, sys: 0 ns, total: 952 ms
Wall time: 979 ms


array([ 1.12448208,  2.0532145 ,  2.91537413,  4.11911927,  5.05015289,
        5.905056  ,  6.97398562,  8.00099195,  8.86229107,  9.9862026 ,
       10.90543608])

In [6]:
%time theta_hat1 = gradient_descent(dJ_math,y,Xb,theta,eta)
theta_hat1

CPU times: user 124 ms, sys: 0 ns, total: 124 ms
Wall time: 142 ms


array([ 1.12448208,  2.0532145 ,  2.91537413,  4.11911927,  5.05015289,
        5.905056  ,  6.97398562,  8.00099195,  8.86229107,  9.9862026 ,
       10.90543608])

通过运行时间可以看出,通过定义求导显然要比通过（分析计算得出）梯度表达式的运算复杂度高，更耗时间。
因此，通过定义求导只能起到一个调试梯度的作用，并不能用于实际的训练中